<a href="https://colab.research.google.com/github/sohdacafe/Machine_Learning/blob/main/kaggle_titanic_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
filename = 'train.csv'
_df = pd.read_csv(filename)
df = _df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [49]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [50]:
# カテゴリ変数に変換
df['Sex'] = pd.get_dummies(df['Sex']).drop('female', axis=1)
df_emb = pd.get_dummies(df['Embarked']).drop('Q', axis=1)
df = pd.concat([df, df_emb], axis=1)
df = df.drop('Embarked', axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,S
0,0,3,1,22.0,1,0,7.2500,0,1
1,1,1,0,38.0,1,0,71.2833,1,0
2,1,3,0,26.0,0,0,7.9250,0,1
3,1,1,0,35.0,1,0,53.1000,0,1
4,0,3,1,35.0,0,0,8.0500,0,1
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0,1
887,1,1,0,19.0,0,0,30.0000,0,1
888,0,3,0,NaN,1,2,23.4500,0,1
889,1,1,1,26.0,0,0,30.0000,1,0


In [52]:
from sklearn.model_selection import train_test_split
t = df['Survived'].values
X = df.drop(labels=['Survived'], axis=1)
X_train, X_test,t_train, t_test = train_test_split(X, t, test_size=0.3, random_state=0)

In [53]:
import lightgbm as lgb
trains = lgb.Dataset(X_train, t_train)
valids = lgb.Dataset(X_test, t_test)

In [54]:
params = {'objective':'regression', 'metrics':'mae'}
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=100, early_stopping_rounds=10)

[1]	valid_0's l1: 0.450588
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 0.431559
[3]	valid_0's l1: 0.414864
[4]	valid_0's l1: 0.39961
[5]	valid_0's l1: 0.385347
[6]	valid_0's l1: 0.372588
[7]	valid_0's l1: 0.361764
[8]	valid_0's l1: 0.350608
[9]	valid_0's l1: 0.341135
[10]	valid_0's l1: 0.332769
[11]	valid_0's l1: 0.324765
[12]	valid_0's l1: 0.317063
[13]	valid_0's l1: 0.310236
[14]	valid_0's l1: 0.304506
[15]	valid_0's l1: 0.298526
[16]	valid_0's l1: 0.29368
[17]	valid_0's l1: 0.289013
[18]	valid_0's l1: 0.284526
[19]	valid_0's l1: 0.280995
[20]	valid_0's l1: 0.277491
[21]	valid_0's l1: 0.274377
[22]	valid_0's l1: 0.271966
[23]	valid_0's l1: 0.268548
[24]	valid_0's l1: 0.266101
[25]	valid_0's l1: 0.264107
[26]	valid_0's l1: 0.262781
[27]	valid_0's l1: 0.26118
[28]	valid_0's l1: 0.260507
[29]	valid_0's l1: 0.259715
[30]	valid_0's l1: 0.259079
[31]	valid_0's l1: 0.257621
[32]	valid_0's l1: 0.256584
[33]	valid_0's l1: 0.255754
[34]	valid_0's l1: 0.25533

In [62]:
# 予測値をroundで四捨五入し、int型に変換
pred = model.predict(X_test).round(0)
pred = list(map(int,pred))

In [ ]:
# testファイルからの予測

In [68]:
filename = 'test.csv'
df_test = pd.read_csv(filename)
df_test = df_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
# カテゴリ変数に変換
df_test['Sex'] = pd.get_dummies(df_test['Sex']).drop('female', axis=1)
df_emb = pd.get_dummies(df_test['Embarked']).drop('Q', axis=1)
df_test = pd.concat([df_test, df_emb], axis=1)
df_test = df_test.drop('Embarked', axis=1)

In [69]:
pred = model.predict(df_test).round(0)
pred = list(map(int,pred))

In [71]:
# 予測値入力、提出ファイル出力
df_gen = pd.read_csv('gender_submission.csv')
df_gen['Survived'] =pred
df_gen.to_csv('gender_submission_4.csv', index=False)

In [ ]:
# 相関性の低いカラムを削除
columns = df_train.columns
for column in columns:
    if df_train[column].dtypes == object :
        continue
    else:
        corr_v = df_train.corr()[column]['Survived']
        print(column, corr_v)
        if corr_v < 0.5 and corr_v > -0.5:
            df_train = df_train.drop(column, axis=1)
            drop_col.append(column)
print(drop_col)

PassengerId -0.005006660767066476
Survived 1.0
Pclass -0.33848103596101586
Age -0.07722109457217737
SibSp -0.03532249888573588
Parch 0.08162940708348222
Fare 0.2573065223849618
['Cabin', 'PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [ ]:
df_train

,Survived,Name,Sex,Ticket,Embarked
0,0,"Braund, Mr. Owen Harris",male,A/5 21171,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C
2,1,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,S
4,0,"Allen, Mr. William Henry",male,373450,S
...,...,...,...,...,...
886,0,"Montvila, Rev. Juozas",male,211536,S
887,1,"Graham, Miss. Margaret Edith",female,112053,S
888,0,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,S
889,1,"Behr, Mr. Karl Howell",male,111369,C
